In [1]:
from pprint import pprint
import pandas as pd

In [2]:
FNDDS_PATH = "D:/Github/FRCE/Datasets/USDA-FNDDS2017-2018 FNDDS At A Glance - FNDDS Nutrient Values.xlsx"

In [ ]:
df_fndds_nutrient_values = pd.read_excel(FNDDS_PATH)
df_fndds_nutrient_values.head()
